# EEG & fNIRS Based Alzheimer's Disease Detection

> TODO: Write a description

In [2]:
!pip install -r ./requirements.txt --quiet

> TODO: write details about the signals

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [4]:
file_path = "./csv_folder/Experiment1/RFECV-5secEEGPSD_FullFnirsPSD_FullFnirsTimeDomain_R-C1-C2-N1-N2-V.csv"
data = pd.read_csv(file_path)

In [5]:
class SignalDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.features = dataframe.iloc[:, 2:].values.astype(np.float32)  # Features start from the 3rd column
        self.labels = dataframe['label'].values.astype(np.int64)  # Label column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]  

In [6]:
dataset = SignalDataset(data)

In [7]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [8]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

In [9]:
model = MLPClassifier(input_dim=151, hidden_dim=32, output_dim=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

In [11]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            predicted = torch.max(outputs, 1)[1]
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")

In [19]:
train_model(model, train_loader, criterion, optimizer)

Epoch 1/20, Loss: 2.8325
Epoch 2/20, Loss: 3.1164
Epoch 3/20, Loss: 2.7816
Epoch 4/20, Loss: 3.0394
Epoch 5/20, Loss: 2.8482
Epoch 6/20, Loss: 2.7015
Epoch 7/20, Loss: 3.0857
Epoch 8/20, Loss: 2.8345
Epoch 9/20, Loss: 2.5915
Epoch 10/20, Loss: 2.4916
Epoch 11/20, Loss: 2.5704
Epoch 12/20, Loss: 2.7913
Epoch 13/20, Loss: 2.6158
Epoch 14/20, Loss: 2.5905
Epoch 15/20, Loss: 3.0578
Epoch 16/20, Loss: 2.3593
Epoch 17/20, Loss: 2.2782
Epoch 18/20, Loss: 2.1760
Epoch 19/20, Loss: 2.9273
Epoch 20/20, Loss: 2.1053


In [20]:
test_model(model, test_loader)

Accuracy: 62.07%


In [21]:
torch.save(model, "./model/model.pth")